In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content
!unzip /content/drive/MyDrive/deepsort.zip

Archive:  /content/drive/MyDrive/deepsort.zip
   creating: deepsort/
  inflating: deepsort/.gitattributes  
   creating: deepsort/deep_sort/
   creating: deepsort/deep_sort/configs/
  inflating: deepsort/deep_sort/configs/deep_sort.yaml  
   creating: deepsort/deep_sort/deep/
   creating: deepsort/deep_sort/deep/checkpoint/
 extracting: deepsort/deep_sort/deep/checkpoint/.gitkeep  
  inflating: deepsort/deep_sort/deep/checkpoint/ckpt.t7  
  inflating: deepsort/deep_sort/deep/checkpoint/original_ckpt.t7  
  inflating: deepsort/deep_sort/deep/evaluate.py  
  inflating: deepsort/deep_sort/deep/feature_extractor.py  
  inflating: deepsort/deep_sort/deep/model.py  
  inflating: deepsort/deep_sort/deep/original_model.py  
  inflating: deepsort/deep_sort/deep/test.py  
  inflating: deepsort/deep_sort/deep/train.py  
 extracting: deepsort/deep_sort/deep/__init__.py  
   creating: deepsort/deep_sort/deep/__pycache__/
  inflating: deepsort/deep_sort/deep/__pycache__/feature_extractor.cpython-310

In [ ]:
!pip install ultralytics
import ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
import torch
torch.__version__

'2.1.0+cu118'

In [ ]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

In [ ]:
%cd /content/deepsort
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

/content/deepsort


In [ ]:
video_path = '/content/drive/MyDrive/front.mp4'
output_path = '/content/output.mp4'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))


In [ ]:
unique_track_ids = set()

In [ ]:
model = YOLO("/content/drive/MyDrive/weights/bestv10.pt")

class_names = ['motorbike', 'vehicle']

colors = [(np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)) for j in range(10)]

while ret:
    ret, frame = cap.read()

    if ret:
        results = model(frame, conf=0.35)

        boxes = results[0].boxes
        conf = boxes.conf
        xywh = boxes.xywh
        cls = boxes.cls.tolist()
        conf = conf.detach().cpu().numpy()
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)

        tracker.update(bboxes_xywh,conf, frame)
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 2)

            cv2.putText(frame, f"{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            unique_track_ids.add(track_id)

        _count = len(unique_track_ids)

        cv2.putText(frame, f"Count: {_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 3 vehicles, 2799.3ms
Speed: 4.7ms preprocess, 2799.3ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 vehicles, 4311.4ms
Speed: 5.3ms preprocess, 4311.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 vehicles, 2547.3ms
Speed: 3.8ms preprocess, 2547.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 vehicles, 2536.4ms
Speed: 9.9ms preprocess, 2536.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 vehicles, 2566.7ms
Speed: 3.3ms preprocess, 2566.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 vehicles, 3458.3ms
Speed: 3.3ms preprocess, 3458.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 vehicles, 3433.3ms
Speed: 10.6ms preprocess, 3433.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 vehicles, 2565.5ms
Speed: 4.3ms preprocess, 2565.5ms inferenc

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

save_path = '/content/output.mp4'

compressed_path = "/content/result.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

0

yolo comp.

In [ ]:
!yolo track model=/content/drive/MyDrive/weights/bestv10.pt source=/content/drive/MyDrive/front.mp4

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.7s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs

video 1/1 (1/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 3031.7ms
video 1/1 (2/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 2929.4ms
video 1/1 (3/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 2958.6ms
video 1/1 (4/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 4911.9ms
video 1/1 (5/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 2921.2ms
video 1/1 (6/901) /content/drive/MyDrive/front.mp4: 384x640 2 vehicles, 2972.1ms
video 1/1 (7/901) /c

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

save_path = '/content/deepsort/runs/detect/predict/front.avi'

compressed_path = "/content/resultYolov8.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

0

In [ ]:
from google.colab import files
files.download('/content/resultYolov8.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>